In [1]:
# S&P 500 COMPANIES: SECTOR, LOCATION, AND DIVERSITY ANALYSIS

import pandas as pd     # first we need to import the pandas library for data manipulation

# here we're scraping the S&P 500 companies from Wikipedia
try:
    scraper = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
    print(scraper)

except Exception as e:
    print(f"An error occurred: {e}")  

# for i, table in enumerate(scraper):
#     print("-----")
#     print(i)
#     print(table)

# scraper[0]

# this shows the first table.. even though there's only one table (just practice)
df = scraper[0]     # df = dataframe

df.to_csv("sp500.csv", index=False)  # save the DataFrame to a CSV file, set standard index to False

df_scraped_file = pd.read_csv("sp500.csv")  # read the CSV file into a DataFrame
df_scraped_file

[    Symbol             Security             GICS Sector  \
0      MMM                   3M             Industrials   
1      AOS          A. O. Smith             Industrials   
2      ABT  Abbott Laboratories             Health Care   
3     ABBV               AbbVie             Health Care   
4      ACN            Accenture  Information Technology   
..     ...                  ...                     ...   
498    XYL           Xylem Inc.             Industrials   
499    YUM          Yum! Brands  Consumer Discretionary   
500   ZBRA   Zebra Technologies  Information Technology   
501    ZBH        Zimmer Biomet             Health Care   
502    ZTS               Zoetis             Health Care   

                                GICS Sub-Industry    Headquarters Location  \
0                        Industrial Conglomerates    Saint Paul, Minnesota   
1                               Building Products     Milwaukee, Wisconsin   
2                           Health Care Equipment  North

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [2]:
import pandas as pd     # import pandas library for data manipulation

# we don't have to scrape the site again, because it saves in the Jupyter Notebook directory so we can just read the CSV file directly
# do try and except to handle the case where the file might not exist (this is generally good practice for handling errors)
try:
    df_scraped_file = pd.read_csv("sp500.csv") 
    print("DataFrame loaded successfully from sp500.csv")
except FileNotFoundError:
    print("File not found. Please ensure that sp500.csv exists in the current directory.")
    exit() # we exit if the file is not found

print("\nDataframe Information:")
print(df.info())   # displays information about the DataFrame

print("\nFirst 5 rows of the DataFrame:")
print(df.head())   # displays the first 5 rows of the DataFrame to check if everything loaded correctly

print("\nColumn Names:")
print(df.columns.tolist())  # prints the names of the columns in the DataFrame


DataFrame loaded successfully from sp500.csv

Dataframe Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 503 non-null    object
 1   Security               503 non-null    object
 2   GICS Sector            503 non-null    object
 3   GICS Sub-Industry      503 non-null    object
 4   Headquarters Location  503 non-null    object
 5   Date added             503 non-null    object
 6   CIK                    503 non-null    int64 
 7   Founded                503 non-null    object
dtypes: int64(1), object(7)
memory usage: 31.6+ KB
None

First 5 rows of the DataFrame:
  Symbol             Security             GICS Sector  \
0    MMM                   3M             Industrials   
1    AOS          A. O. Smith             Industrials   
2    ABT  Abbott Laboratories             Health Care   
3

In [3]:
#  --- CLEANING UP COLUMN NAMES ---
# I'm using the chaining method for better readability
# using the parenthesis method allows to continue a long line of code without breaking the chain
# regex is short for regular expressions, which defines a search pattern; in this case, it's set to False

df.columns = (df.columns
                    .str.strip()    # remove any leading or trailing whitespace from column names
                     .str.replace('\n', '', regex=False) # remove newline characters from column names
                     .str.replace('-', '_', regex=False) # replace hyphens with underscores in column names
                     .str.replace(' ', '_', regex=False) # replace spaces with underscores in column names
                     .str.lower()) # convert column names to lowercase for consistency

print("\nCleaned Column Names:")
print(df.columns.tolist()) # checking the new column names


# --- CLEANING DATA WITHIN SPECIFIC COLUMNS -----
# we're using the new and cleaned column names here
columns_to_clean = [
    'security',                 # 'security' is the name of the company
    'symbol',                   # 'symbol' is the stock ticker symbol
    'gics_sector',              # 'gics_sector' is the Global Industry Classification Standard sector
    'gics_sub_industry',        # 'gics_sub_industry' is the sub-industry classification
    'headquarters_location'     # 'headquarters_location' is the location of the company's headquarters
]

for col_name in columns_to_clean:
    print(f"Cleaning column: {col_name}")

    # for each column in the list, we will clean it up by removing citations, whitespace, and converting 'nan' strings to actual NaN values
    # we check if the column exists in the DataFrame before attempting to clean it (helps to avoid errors)
    if col_name in df.columns:
        df[col_name] = (df[col_name]
                    .astype(str)     # convert to string
                
                    .str.replace(r'\[.*?\]', '', regex=True)     # remove [citations]; this means "find a literal opening square bracket, followed by any characters 
                                                                    # (zero or more times, but as few as possible), followed by a literal closing square bracket"

                    .str.strip()                 # remove whitespace
                    .replace('nan', pd.NA))     # fix string 'nan' back to actual NaN (NaN = not a number)
    else:
        print(f"Warning: Column '{col_name}' not found for data cleaning.")

print("\nDataFrame after basic data cleaning (first 5 rows): ")
print(df.head())





Cleaned Column Names:
['symbol', 'security', 'gics_sector', 'gics_sub_industry', 'headquarters_location', 'date_added', 'cik', 'founded']
Cleaning column: security
Cleaning column: symbol
Cleaning column: gics_sector
Cleaning column: gics_sub_industry
Cleaning column: headquarters_location

DataFrame after basic data cleaning (first 5 rows): 
  symbol             security             gics_sector  \
0    MMM                   3M             Industrials   
1    AOS          A. O. Smith             Industrials   
2    ABT  Abbott Laboratories             Health Care   
3   ABBV               AbbVie             Health Care   
4    ACN            Accenture  Information Technology   

                gics_sub_industry    headquarters_location  date_added  \
0        Industrial Conglomerates    Saint Paul, Minnesota  1957-03-04   
1               Building Products     Milwaukee, Wisconsin  2017-07-26   
2           Health Care Equipment  North Chicago, Illinois  1957-03-04   
3              

In [ ]:
import pandas as pd     # import pandas library for data manipulation

import numpy as np      # import numpy library for numerical operations
# NumPy = Numerical Python, a library for for scientific computing with Python
# provides fast, efficient array operations and mathematical operations

# --- SPLITTING HEADQUARTERS LOCATION (ONELINER) INTO CITY AND STATE ---
def split_headquarters_oneliner(df):
    clean_location = df['headquarters_location'].fillna('').astype(str)     # this is taking the column 'headquarters_location', filling NaN values with an empty string, converting it to string type
    split_cols = clean_location.str.rsplit(',', n=1, expand=True)           # right splits the string at the last comma, perform at most one split, and expands the result into separate columns 
                                                                            # if Expand = false it would return a Series where each element is a list of the split parts    
    df['headquarters_city'] = split_cols[0].str.strip()
    df['headquarters_state'] = split_cols[1].str.strip().fillna('') if split_cols.shape[1] > 1 else ''
    
    return df

# COMPLETE EXAMPLE WITH YOUR DATA:
# Apply the optimized function
df = split_headquarters_oneliner(df)

# Display results
print("DataFrame after splitting headquarters location:")
print(df[['security','symbol', 'gics_sector', 'gics_sub_industry','headquarters_location', 'headquarters_city', 'headquarters_state']].head())

print("\nUnique states found (top 10):")
print(df['headquarters_state'].value_counts().head(10))
